Esse notbook tem o intuito de treinar os dados de forma nao supervisionada, criando custering para identificar padroes em bairros do crime de trafico de drogas e alertar quais estao crescendo ou nao.

Importando bibliotecas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

Lendo o arquivo

In [ ]:
df = pd.read_csv("C:/Users/55819/Downloads/Crime_prediction/data/processed/dados_processados.csv")
df.head(10)

In [ ]:
df_trafico = df[df['tipo_crime'].str.contains('tráfico', case=False, na=False)].copy()

In [ ]:
df_trafico['tem_arma'] = np.where(df_trafico['arma_utilizada'].str.lower() != 'não informado', 1, 0)


In [ ]:
df_bairros = df_trafico.groupby('bairro').agg({
    'quantidade_suspeitos': 'mean',
    'quantidade_vitimas': 'mean',
    'tem_arma': 'mean',  # percentual de ocorrências com arma
    'idade_suspeito': 'mean',
    'hora': 'mean'       # horário médio das ocorrências
}).reset_index()

# Renomear colunas
df_bairros.rename(columns={
    'quantidade_suspeitos': 'media_suspeitos',
    'quantidade_vitimas': 'media_vitimas',
    'tem_arma': 'percentual_com_arma',
    'idade_suspeito': 'media_idade_suspeitos',
    'hora': 'media_hora'
}, inplace=True)

In [ ]:
df_bairros.fillna(df_bairros.mean(numeric_only=True), inplace=True)

scaler = StandardScaler()
X = scaler.fit_transform(df_bairros.drop(columns='bairro'))

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
df_bairros['cluster'] = kmeans.fit_predict(X)


silhouette = silhouette_score(X_scaled, df_bairros['cluster'])
davies_bouldin = davies_bouldin_score(X_scaled, df_bairros['cluster'])
calinski = calinski_harabasz_score(X_scaled, df_bairros['cluster'])

print(f"Silhouette Score: {silhouette:.3f}")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f}")
print(f"Calinski-Harabasz Index: {calinski:.3f}")

In [ ]:
cluster_summary = df_bairros.groupby('cluster').mean(numeric_only=True).round(2)
print("MÉDIAS DOS CLUSTERS:")
print(cluster_summary)

In [ ]:
print("\nRESUMO AUTOMÁTICO DOS CLUSTERS:\n")

for c in sorted(df_bairros['cluster'].unique()):
    grupo = df_bairros[df_bairros['cluster'] == c]
    qtd_bairros = grupo.shape[0]
    media_suspeitos = grupo['media_suspeitos'].mean().round(2)
    perc_arma = (grupo['percentual_com_arma'].mean() * 100).round(1)
    hora_media = grupo['media_hora'].mean().round(1)
    idade_media = grupo['media_idade_suspeitos'].mean().round(1)

    # gerar “interpretação automática”
    interpretacao = []
    if perc_arma > 40:
        interpretacao.append("tráfico armado")
    else:
        interpretacao.append("tráfico sem arma")

    if media_suspeitos >= 2:
        interpretacao.append("com vários suspeitos (organizado)")
    else:
        interpretacao.append("isolado (poucos suspeitos)")

    if hora_media >= 18:
        interpretacao.append("ocorrendo mais à noite")
    elif hora_media <= 6:
        interpretacao.append("ocorrendo de madrugada")
    else:
        interpretacao.append("ocorrendo durante o dia")

    print(f"Cluster {c} ({qtd_bairros} bairros):")
    print(f"Média de suspeitos: {media_suspeitos}")
    print(f"{perc_arma}% dos casos com arma")
    print(f"Idade média dos suspeitos: {idade_media} anos")
    print(f"Horário médio das ocorrências: {hora_media}h")
    print(f"Perfil: {', '.join(interpretacao)}")
    print(f"Bairros: {', '.join(grupo['bairro'].to_list())}")
    print("-" * 80)

In [ ]:

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Pegar apenas as features numéricas padronizadas
from sklearn.preprocessing import StandardScaler

features = ['media_suspeitos', 'media_vitimas', 'percentual_com_arma', 'media_idade_suspeitos', 'media_hora']
X = df_bairros[features]
X_scaled = StandardScaler().fit_transform(X)

# Reduzir para 2 dimensões
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

# Adicionar resultados ao dataframe
df_bairros['PCA1'] = pca_result[:, 0]
df_bairros['PCA2'] = pca_result[:, 1]

# Plotar os clusters agrupados
plt.figure(figsize=(9,7))
sns.scatterplot(
    data=df_bairros,
    x='PCA1',
    y='PCA2',
    hue='cluster',
    palette='viridis',
    s=200,
    alpha=0.8
)
for i, row in df_bairros.iterrows():
    plt.text(row['PCA1'] + 0.05, row['PCA2'] + 0.05, row['bairro'], fontsize=9)

plt.title('Agrupamento de Bairros por Perfil de Tráfico (PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.show()
